In [1]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Prepare Column Names

In [2]:
dtypes = {
 'Date First Observed': np.str,
 'Days Parking In Effect    ': np.str,
 'Double Parking Violation': np.str,
 'Feet From Curb': np.float32,
 'From Hours In Effect': np.str,
 'House Number': np.str,
 'Hydrant Violation': np.str,
 'Intersecting Street': np.str,
 'Issue Date': np.str,
 'Issuer Code': np.float32,
 'Issuer Command': np.str,
 'Issuer Precinct': np.float32,
 'Issuer Squad': np.str,
 'Issuing Agency': np.str,
 'Law Section': np.float32,
 'Meter Number': np.str,
 'No Standing or Stopping Violation': np.str,
 'Plate ID': np.str,
 'Plate Type': np.str,
 'Registration State': np.str,
 'Street Code1': np.uint32,
 'Street Code2': np.uint32,
 'Street Code3': np.uint32,
 'Street Name': np.str,
 'Sub Division': np.str,
 'Summons Number': np.uint32,
 'Time First Observed': np.str,
 'To Hours In Effect': np.str,
 'Unregistered Vehicle?': np.str,
 'Vehicle Body Type': np.str,
 'Vehicle Color': np.str,
 'Vehicle Expiration Date': np.str,
 'Vehicle Make': np.str,
 'Vehicle Year': np.float32,
 'Violation Code': np.uint16,
 'Violation County': np.str,
 'Violation Description': np.str,
 'Violation In Front Of Or Opposite': np.str,
 'Violation Legal Code': np.str,
 'Violation Location': np.str,
 'Violation Post Code': np.str,
 'Violation Precinct': np.float32,
 'Violation Time': np.str
}

# Read in CSV file

In [3]:
path = 'C:/Users/phan/OneDrive - adesso Group/DataSet/nyc-parking-tickets/*.csv'
df = dd.read_csv(urlpath=path, dtype=dtypes, usecols=dtypes.keys())
df

Dask DataFrame Structure:
                Summons Number Plate ID Registration State Plate Type Issue Date Violation Code Vehicle Body Type Vehicle Make Issuing Agency Street Code1 Street Code2 Street Code3 Vehicle Expiration Date Violation Location Violation Precinct Issuer Precinct Issuer Code Issuer Command Issuer Squad Violation Time Time First Observed Violation County Violation In Front Of Or Opposite House Number Street Name Intersecting Street Date First Observed Law Section Sub Division Violation Legal Code Days Parking In Effect     From Hours In Effect To Hours In Effect Vehicle Color Unregistered Vehicle? Vehicle Year Meter Number Feet From Curb Violation Post Code Violation Description No Standing or Stopping Violation Hydrant Violation Double Parking Violation
npartitions=142                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
                        uint32   object             object     object     object         uint16            object       object         object       uint32       uint32       uint32                  object             object            float32         float32     float32         object       object         object              object           object                            object       object      object              object              object     float32       object               object                     object               object             object        object                object      float32       object        float32              object                object                            object            object                   object
                           ...      ...                ...        ...        ...            ...               ...          ...            ...          ...          ...          ...                     ...                ...                ...             ...         ...            ...          ...            ...                 ...              ...                               ...          ...         ...                 ...                 ...         ...          ...                  ...                        ...                  ...                ...           ...                   ...          ...          ...            ...                 ...                   ...                               ...               ...                      ...
...                        ...      ...                ...        ...        ...            ...               ...          ...            ...          ...          ...          ...                     ...                ...                ...             ...         ...            ...          ...            ...                 ...              ...                               ...          ...         ...                 ...                 ...         ...          ...                  ...                        ...                  ...                ...           ...                   ...          ...          ...            ...                 ...                   ...                               ...               ...                      ...
                           ...      ...                ...        ...        ...            ...               ...          ...            ...          ...          ...          ..

In [5]:
pathParquet = 'C:/Users/phan/OneDrive - adesso Group/DataSet/nyc-parking-tickets/nyc_final'
with ProgressBar():
    df.to_parquet(path=pathParquet)

[########################################] | 100% Completed |  7min 22.5s


# Select a single Column

In [31]:
with ProgressBar():
    display(df['Plate ID'].head())


[########################################] | 100% Completed |  1.8s


0    GBB9093
1    62416MB
2    78755JZ
3    63009MA
4    91648MC
Name: Plate ID, dtype: object

# Compute Missing Percentage

In [6]:
missingValues = df.isnull().sum()
with ProgressBar():
    percentMissing = ((missingValues / df.index.size) * 100).compute()
percentMissing

[########################################] | 100% Completed |  4min  0.3s


Summons Number                        0.000000
Plate ID                              0.020867
Registration State                    0.000000
Plate Type                            0.000000
Issue Date                            0.000000
Violation Code                        0.000000
Vehicle Body Type                     0.564922
Vehicle Make                          0.650526
Issuing Agency                        0.000000
Street Code1                          0.000000
Street Code2                          0.000000
Street Code3                          0.000000
Vehicle Expiration Date               0.000002
Violation Location                   15.142846
Violation Precinct                    0.000002
Issuer Precinct                       0.000002
Issuer Code                           0.000002
Issuer Command                       15.018851
Issuer Squad                         15.022566
Violation Time                        0.019207
Time First Observed                  90.040886
Violation Cou

# Drop columns with more than 50% data missing

In [5]:
dfCleanStage1 = df.drop(labels=list(percentMissing[percentMissing >= 50].index), axis=1)

KeyboardInterrupt: 

In [45]:
with ProgressBar():
    countOfVehicleColors = dfCleanStage1['Vehicle Color'].value_counts().compute()
mostCommonColor = countOfVehicleColors.sort_values(ascending=False).index[0]

dfCleanStage2 = dfCleanStage1.fillna(value={'Vehicle Color': mostCommonColor})

[########################################] | 100% Completed |  3min 33.7s


In [48]:
rowsToDrop = list(percentMissing[(percentMissing > 0) & (percentMissing < 5)].index)
dfCleanStage3 = dfCleanStage2.dropna(subset=rowsToDrop)

In [65]:
remainingColumnsToClean = list(percentMissing[(percentMissing >= 5) & (percentMissing < 50)].index)
df.dtypes[remainingColumnsToClean]

Violation Location                   object
Issuer Command                       object
Issuer Squad                         object
Violation County                     object
Violation In Front Of Or Opposite    object
House Number                         object
Days Parking In Effect               object
From Hours In Effect                 object
To Hours In Effect                   object
Violation Post Code                  object
Violation Description                object
dtype: object

In [72]:
unknownDefaultDict = dict(map(lambda columnName: (columnName, 'Unknown'), remainingColumnsToClean))
unknownDefaultDict

{'Violation Location': 'Unknown',
 'Issuer Command': 'Unknown',
 'Issuer Squad': 'Unknown',
 'Violation County': 'Unknown',
 'Violation In Front Of Or Opposite': 'Unknown',
 'House Number': 'Unknown',
 'Days Parking In Effect    ': 'Unknown',
 'From Hours In Effect': 'Unknown',
 'To Hours In Effect': 'Unknown',
 'Violation Post Code': 'Unknown',
 'Violation Description': 'Unknown'}

In [71]:
{columnName: 'Unknown' for columnName in remainingColumnsToClean}

{'Violation Location': 'Unknown',
 'Issuer Command': 'Unknown',
 'Issuer Squad': 'Unknown',
 'Violation County': 'Unknown',
 'Violation In Front Of Or Opposite': 'Unknown',
 'House Number': 'Unknown',
 'Days Parking In Effect    ': 'Unknown',
 'From Hours In Effect': 'Unknown',
 'To Hours In Effect': 'Unknown',
 'Violation Post Code': 'Unknown',
 'Violation Description': 'Unknown'}

In [73]:
dfCleanStage4 = dfCleanStage3.fillna(value=unknownDefaultDict)

In [74]:
with ProgressBar():
    print(dfCleanStage4.isnull().sum().compute())
dfCleanStage4.persist()

[########################################] | 100% Completed |  5min 34.8s
Summons Number                       0
Plate ID                             0
Registration State                   0
Plate Type                           0
Issue Date                           0
Violation Code                       0
Vehicle Body Type                    0
Vehicle Make                         0
Issuing Agency                       0
Street Code1                         0
Street Code2                         0
Street Code3                         0
Vehicle Expiration Date              0
Violation Location                   0
Violation Precinct                   0
Issuer Precinct                      0
Issuer Code                          0
Issuer Command                       0
Issuer Squad                         0
Violation Time                       0
Violation County                     0
Violation In Front Of Or Opposite    0
House Number                         0
Street Name                  

Dask DataFrame Structure:
                Summons Number Plate ID Registration State Plate Type Issue Date Violation Code Vehicle Body Type Vehicle Make Issuing Agency Street Code1 Street Code2 Street Code3 Vehicle Expiration Date Violation Location Violation Precinct Issuer Precinct Issuer Code Issuer Command Issuer Squad Violation Time Violation County Violation In Front Of Or Opposite House Number Street Name Date First Observed Law Section Sub Division Days Parking In Effect     From Hours In Effect To Hours In Effect Vehicle Color Vehicle Year Feet From Curb Violation Post Code Violation Description
npartitions=142                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
                        uint32   object             object     object     object         uint16            object       object         object       uint32       uint32       uint32                  object             object            float32         float32     float32         object       object         object           object                            object       object      object              object     float32       object                     object               object             object        object      float32        float32              object                object
                           ...      ...                ...        ...        ...            ...               ...          ...            ...          ...          ...          ...                     ...                ...                ...             ...         ...            ...          ...            ...              ...                               ...          ...         ...                 ...         ...          ...                        ...                  ...                ...           ...          ...            ...                 ...                   ...
...                        ...      ...                ...        ...        ...            ...               ...          ...            ...          ...          ...          ...                     ...                ...                ...             ...         ...            ...          ...            ...              ...                               ...          ...         ...                 ...         ...          ...                        ...                  ...                ...           ...          ...            ...                 ...                   ...
                           ...      ...                ...        ...        ...            ...               ...          ...            ...          ...          ...          ...                     ...                ...                ...             ...         ...            ...          ...            ...              ...                               ...          ...         ...                 ...         ...          ...                        ...                  ...                ...           ...          ...            ...                 ...                   ...
                           ...      ...                ...        ...        ...            ...               ...          ...            ...          ...          ...          ...                     ...                ...                ...             ...         ...            ...          ...            ...              ...                               ...          ...         ...                 ...         ...          ...                        

In [78]:
dfCleanStage4.head()

Summons Number Plate ID Registration State Plate Type  Issue Date  \
0      1283294138  GBB9093                 NY        PAS  08/04/2013   
1      1283294151  62416MB                 NY        COM  08/04/2013   
2      1283294163  78755JZ                 NY        COM  08/05/2013   
3      1283294175  63009MA                 NY        COM  08/05/2013   
4      1283294187  91648MC                 NY        COM  08/08/2013   

   Violation Code Vehicle Body Type Vehicle Make Issuing Agency  Street Code1  \
0              46              SUBN         AUDI              P         37250   
1              46               VAN         FORD              P         37290   
2              46               P-U        CHEVR              P         37030   
3              46               VAN         FORD              P         37270   
4              41              TRLR          GMC              P         37240   

   ...  Law Section  Sub Division Days Parking In Effect      \
0  ...        408.0            F1                    BBBBBBB   
1  ...        408.0             C                    BBBBBBB   
2  ...        408.0            F7                    BBBBBBB   
3  ...        408.0            F1                    BBBBBBB   
4  ...        408.0            E1                    BBBBBBB   

  From Hours In Effect  To Hours In Effect  Vehicle Color  Vehicle Year  \
0                  ALL                 ALL             GY        2013.0   
1                  ALL                 ALL             WH        2012.0   
2                  ALL                 ALL             GY           0.0   
3                  ALL                 ALL             WH        2010.0   
4                  ALL                 ALL             BR        2012.0   

  Feet From Curb Violation Post Code Violation Description  
0            0.0             Unknown               Unknown  
1            0.0             Unknown               Unknown  
2            0.0             Unknown               Unknown  
3            0.0             Unknown               Unknown  
4            0.0             Unknown               Unknown  

[5 rows x 35 columns]

In [81]:
dfCleanStage4['Plate Type'].value_counts().compute()

KeyboardInterrupt: 